### data load

In [25]:
import pandas as pd
# 파일 이름/경로는 환경에 맞게
df = pd.read_csv("C:/Users/Admin/Desktop/PROJ/data/01_raw/fra_cleaned.csv", encoding="cp1252", sep=";")

### gender & brand 필터링

In [26]:
target_brands = [
    "diptyque",
    "lanvin",
    "byredo",
    "yvessaintlaurent",
    "santamarianovella",
    "lelabo",
    "maisonfranciskurkdjian",
    "chanel",
    "acquadiparma",
    "jomalonelondon",
    "jimmychoo",
    "guerlain",
    "calvinklein",
    "arianagrande",
    "versace",
    "exnihilo",
    "memoparis",
    "demeterfragrance",
    "elizabetharden",
    "creed",
    "giorgioarmani",
]

# 브랜드 정리
def is_target_brand(brand: str) -> bool:
    if pd.isna(brand):
        return False
    norm = str(brand).replace("-", "").lower()
    return any(t in norm for t in target_brands)

mask = df["Brand"].apply(is_target_brand)

df_filtered = df[mask].copy()

# 성별 정리
filtered_df = df_filtered[df_filtered["Gender"].str.lower().isin(["women", "unisex"])]

In [27]:
# 향수 분석에 필요없는 컬럼 제거
filtered_df = filtered_df.drop(columns=['Country','Year','Perfumer1','Perfumer2'], axis=1)

### perfume_id(PK) 추가

In [28]:
# 인덱스 정렬하고 인덱스를 컬럼으로 추가(PK)
df_final = filtered_df.reset_index(drop=True)
df_final = df_final.reset_index().rename(columns={"index": "perfume_id"})

In [29]:
# df_final.to_csv("C:/Users/Admin/Desktop/PROJ/data/02_cleaned/0.perfume_gender_brand_filtered.csv",
#     index=False,
#     encoding="utf-8-sig"
# )

### (향수 id & url만 파일 생성)

In [30]:
# 향수 id & url만 
df_id = df_final.iloc[:,:2]

In [31]:
# df_id.to_csv("C:/Users/Admin/Desktop/PROJ/data/02_cleaned/perfume_id_url.csv",
#     index=False,
#     encoding="utf-8-sig"
# )

### mainacccord 결측치 처리

36개의 행 추가 크롤링으로 처리

In [32]:
file_path = 'C:/Users/Admin/Desktop/PROJ/data/02_cleaned/perfume/1.perfume_mainaccord_missing_filled.csv'
df_clean = pd.read_csv(file_path)

### 한정판 향수 제거

In [33]:
#limited-edition 포함된 행 지우기
filtered_df = df_clean[~df_clean['Perfume'].str.contains('limited-edition', case=False, na=False)]

### mainaccord 이상치 처리(희귀향) 

In [34]:
accord_cols = ['mainaccord1', 'mainaccord2', 'mainaccord3',]

# 소문자로 변환 & 앞뒤 공백 제거
for col in accord_cols:
    filtered_df[col] = filtered_df[col].astype(str).str.lower().str.strip()

# 기준: 전체에서 20번 미만 등장하면 삭제
threshold = 20
loop_count = 0
while True:
    loop_count += 1

    # 1. 현재 상태에서 빈도수 계산
    all_accords = filtered_df[accord_cols].stack()
    accord_counts = all_accords.value_counts()

    # 2. 기준 미만인 향 찾기
    rare_accords = set(accord_counts[accord_counts < threshold].index)

    # 3. 더 이상 지울 게 없으면 멈춤
    if len(rare_accords) == 0:
        break

    # 4. 희귀 향이 하나라도 포함된 행 삭제
    condition_mask = filtered_df[accord_cols].isin(rare_accords).any(axis=1)
    filtered_df = filtered_df[~condition_mask]

# 저장
filtered_df.to_csv('C:/Users/Admin/Desktop/PROJ/data/02_cleaned/perfume/2.perfume_limited_mainaccord_filtered.csv', index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\1798286842.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col] = filtered_df[col].astype(str).str.lower().str.strip()
C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\1798286842.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col] = filtered_df[col].astype(str).str.lower().str.strip()
C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\1798286842.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f